In [23]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel
import json

In [24]:
def load_data(data):
	df = pd.read_csv(data)
	return df 

In [25]:
def vectorize_text_to_cosine_mat(data):
	count_vect = CountVectorizer()
	cv_mat = count_vect.fit_transform(data)
 
	cosine_sim_mat = cosine_similarity(cv_mat)
	return cosine_sim_mat

In [26]:
def get_recommendation(title,cosine_sim_mat,df,num_of_rec=10):

	course_indices = pd.Series(df.index,index=df['course_title']).drop_duplicates()

	idx = course_indices[title]

	sim_scores =list(enumerate(cosine_sim_mat[idx]))
	sim_scores = sorted(sim_scores,key=lambda x: x[1],reverse=True)
	selected_course_indices = [i[0] for i in sim_scores[1:]]
	selected_course_scores = [i[0] for i in sim_scores[1:]]

	result_df = df.iloc[selected_course_indices]
	result_df['similarity_score'] = selected_course_scores
	final_recommended_courses = result_df[['course_title','similarity_score','url','price','num_subscribers']]
	return final_recommended_courses.head(num_of_rec)

In [27]:
def search_term_if_not_found(term,df):
	result_df = df[df['course_title'].str.contains(term)]
	return result_df

In [28]:
def predict_course(search_term):
  df = load_data("udemy_course_data.csv")
  cosine_sim_mat = vectorize_text_to_cosine_mat(df['course_title'])
  num_of_rec = 7
  if search_term is not None:
        try:
          results = get_recommendation(search_term,cosine_sim_mat,df,num_of_rec)
          jresult = json.dump(results)
          return jresult
        except:
          results= "Not Found"
          result_df = search_term_if_not_found(search_term,df)
          result_df = result_df.to_json()
          jresultx = json.dumps(result_df)
          return jresultx


      

In [ ]:
result = predict_course("Stock")
print(result)

In [30]:
import pickle

with open('predict_course.pkl', 'wb') as f:
    pickle.dump(predict_course, f)


In [31]:
model = pickle.load(open('predict_course.pkl', 'rb'))
model.predict_course("Stock")

AttributeError: 'function' object has no attribute 'predict_course'